# 0. S3バケットの作成

あらかじめS3バケットを作成しておく。

In [1]:
bucket = 'speg03-sagemaker'

# 1. SageMakerセッションの作成

In [2]:
from sagemaker import Session, get_execution_role

In [3]:
session = Session()
role = get_execution_role()

# 2. データセットの準備

In [4]:
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

In [5]:
(x_train, y_train), (x_test, y_test) = load_data()

!mkdir -p ./data/cifar10/{train,val}
np.savez('./data/cifar10/train/cifar10_train.npz', x=x_train, y=y_train)
np.savez('./data/cifar10/val/cifar10_val.npz', x=x_test, y=y_test)

In [6]:
train_data = session.upload_data('./data/cifar10/train', bucket=bucket, key_prefix='data/cifar10/train')
train_data

's3://speg03-sagemaker/data/cifar10/train'

In [7]:
val_data = session.upload_data('./data/cifar10/val', bucket=bucket, key_prefix='data/cifar10/val')
val_data

's3://speg03-sagemaker/data/cifar10/val'

# 2. ローカルモードで学習

In [8]:
from sagemaker.tensorflow import TensorFlow

In [9]:
estimator = TensorFlow(
    role=role,
    source_dir='src',
    entry_point='train.py',
    train_instance_count=1,
    train_instance_type='local',
    framework_version='1.12.0',
    py_version='py3',

    hyperparameters=dict(
        limit_data_rate=0.01
    ),

    code_location=f's3://{bucket}/local_output',
    output_path=f's3://{bucket}/local_output'
)

In [10]:
estimator.fit(dict(train=train_data, validation=val_data))

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-03-13-15-26-09-314


Creating tmp3cqe9hh7_algo-1-ynxr6_1 ... 
Attaching to tmp3cqe9hh7_algo-1-ynxr6_1
algo-1-ynxr6_1  | 2019-03-13 15:26:24,603 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-ynxr6_1  | 2019-03-13 15:26:24,629 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ynxr6_1  | 2019-03-13 15:26:24,962 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ynxr6_1  | 2019-03-13 15:26:25,076 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ynxr6_1  | 2019-03-13 15:26:25,101 sagemaker-containers INFO     Invoking user script
algo-1-ynxr6_1  | 
algo-1-ynxr6_1  | Training Env:
algo-1-ynxr6_1  | 
algo-1-ynxr6_1  | {
algo-1-ynxr6_1  |     "additional_framework_parameters": {},
algo-1-ynxr6_1  |     "channel_input_dirs": {
algo-1-ynxr6_1  |         "train": "/opt/ml/input/data/train",
algo-1-ynxr6_1  |         "validation": "/opt/ml/input/data/validation"
algo-1